# Compare spectra produced by two different continuum subtraction techniques

This notebook compares the spectra produced with two continuum substraction techniques: in the uv plane and in the image plane.

## Initialization

In [ ]:
!pip install -r requirements.txt > /dev/null

In [ ]:
import numpy as np
from astropy import units as u
from spectral_cube import SpectralCube
from matplotlib import pyplot as plt
from compass import datadir, extract_spectrum_from_cube, reference_position

In [ ]:
%matplotlib widget

## Define the source, the frequency setting and the spectral window

In [ ]:
source = "bhr71"
pos = reference_position[source]
setting = 5
spw = 27
test_subdir = "tests/contsub/"

## List of frequencies with potential baseline substraction problems in the UV plane

See this [document](https://docs.google.com/spreadsheets/d/1l571cEiDvEaeWhq0sk4l4viM7bEgSS-csxVGiSgsM0E/edit?gid=0#gid=0).

In [ ]:
freq_with_baseline_problem = [
    289.780,
    291.022,
    283.200,
    286.081,
    299.533,
    309.896,
    309.876,
    283.398,
    288.211,
    286.570,
    289.664,
    294.583,
    295.913,
    296.880,
    305.663,
    279.450,
    294.375,
    294.75,
    285.625,
    285.450,
    286.6,
    303.250,
    280.100,
    279.97,
    296.44,
    294.8,
    294.96,
    295.18,
    303.160,
    303.250,
    285.43,
    295.21,
    294.960,
    285.77,
    303.67,
]

## Read the cube produced with a substraction in the UV plane

In [ ]:
cube_fid = SpectralCube.read(
    datadir + "{}/cubes/{}-set{}-spw{}-lines.fits".format(source, source, setting, spw)
)
restfreq_fid = cube_fid.header["RESTFRQ"] * u.Hz
spectrum_fid = extract_spectrum_from_cube(cube_fid, pos)

## Read the cube produced with a substraction in the image plane

In [ ]:
cube_test = SpectralCube.read(
    datadir
    + "{}/{}/cubes/{}-set{}-spw{}-lines.fits".format(
        test_subdir, source, source, setting, spw
    )
)
restfreq_test = cube_test.header["RESTFRQ"] * u.Hz
spectrum_test = extract_spectrum_from_cube(cube_test, pos)

## Plot the spectra

In [ ]:
fig, ax = plt.subplots(
    num="{} setting {} spw {}".format(source, setting, spw), clear=True
)
f = ax.step(
    spectrum_fid.spectral_axis.to(
        u.GHz, equivalencies=u.doppler_radio(restfreq_fid)
    ).value,
    spectrum_fid.to(u.K).value,
    label="UV plane",
)
t = ax.step(
    spectrum_test.spectral_axis.to(
        u.GHz, equivalencies=u.doppler_radio(restfreq_test)
    ).value,
    spectrum_test.to(u.K).value,
    label="Image plane",
)
baseline = np.zeros_like(spectrum_fid.spectral_axis.value)
b = ax.plot(
    spectrum_fid.spectral_axis.to(
        u.GHz, equivalencies=u.doppler_radio(restfreq_fid)
    ).value,
    baseline,
    color="black",
    linestyle="dotted",
)
ax.set_xlabel("Frequency (GHz)")
ax.set_ylabel("Brightness temperature (K)")
ll = ax.legend()

## Mark the regions with potential baseline substraction problems

In [ ]:
bandwidth = 0.005
fmin, fmax = ax.get_xlim()
for freq in freq_with_baseline_problem:
    if freq > fmin and freq < fmax:
        ax.axvspan(
            freq - bandwidth / 2.0,
            freq + bandwidth / 2.0,
            color="red",
            alpha=0.25,
            lw=0,
        )

## Show the RMS

In [ ]:
rms = (cube_fid.header["NOISEMEA"] * cube_fid.unit).to(
    u.K, u.brightness_temperature(restfreq_fid, cube_fid.beam)
)
ax.axhspan(
    -rms.value,
    rms.value,
    color="grey",
    alpha=0.25,
)  # +/- 1 sigma